In [ ]:
import pandas as pd
import sqlalchemy

In [ ]:
engine = sqlalchemy.create_engine('******')

In [ ]:
enox="%%ENOXAPARIN%%"
hep="%%HEPARIN%%"
query = f'''
select * from (
with cohort as
    --patients who got LMWH or SC Heparin within 24 hours at least once
    (  
        select icustays.stay_id, icustays.hadm_id, icustays.intime
        from mimic_icu.icustays  icustays
        inner join mimic_hosp.emar emar
        on icustays.hadm_id=emar.hadm_id
        inner join  mimic_hosp.pharmacy pharmacy
        on pharmacy.pharmacy_id=emar.pharmacy_id
        where event_txt = 'Administered'
        and icustays.intime<=emar.scheduletime
        and datetime_add(icustays.intime, interval '24 hours')  >= emar.scheduletime
        and (
           upper(emar.medication) like '{enox}'
           or
          (upper(emar.medication) like '{hep}' and pharmacy.route = 'SC')
            )
        group by icustays.stay_id, icustays.hadm_id, icustays.intime
    )
--blood gases
    select cohort.stay_id,cohort.intime,bg.charttime,'ph' as event, bg.ph event_value
    from  cohort
    inner join mimic_derived.bg bg
    on bg.hadm_id=cohort.hadm_id
    where bg.ph is not null
union all 
    select cohort.stay_id,cohort.intime,bg.charttime,'baseexcess' as event, bg.baseexcess as event_value
    from  cohort
    inner join mimic_derived.bg bg
    on bg.hadm_id=cohort.hadm_id
    where  bg.baseexcess is not null
union all 
    select cohort.stay_id,cohort.intime,bg.charttime,'lactate' as event, bg.lactate as event_value
    from  cohort
    inner join mimic_derived.bg bg
    on bg.hadm_id=cohort.hadm_id
    where  bg.lactate is not null
union all 
--coagulation
    select cohort.stay_id, cohort.intime, coagulation.charttime, 'inr' as event, coagulation.inr as event_value
    from  cohort
    inner join mimic_derived.coagulation coagulation
    on coagulation.hadm_id=cohort.hadm_id
    and cohort.intime<=coagulation.charttime
    where coagulation.inr is not null
union all 
    select cohort.stay_id, cohort.intime, coagulation.charttime,'pt' as event, coagulation.pt as event_value
    from  cohort 
    inner join mimic_derived.coagulation coagulation
    on coagulation.hadm_id=cohort.hadm_id
    and cohort.intime<=coagulation.charttime
    where coagulation.pt is not null
union all 
    select cohort.stay_id,cohort.intime, coagulation.charttime,'ptt' as event, coagulation.ptt  as event_value
    from  cohort
    inner join mimic_derived.coagulation coagulation
    on coagulation.hadm_id=cohort.hadm_id
    and cohort.intime<=coagulation.charttime
    where coagulation.ptt is not null
union all 
--chemistry
    select cohort.stay_id, cohort.intime, chemistry.charttime, 'albumin' as event, chemistry.albumin as event_value
    from cohort
    inner join mimic_derived.chemistry chemistry
    on chemistry.hadm_id=cohort.hadm_id
    and cohort.intime<=chemistry.charttime
    where chemistry.albumin is not null
union all 
    select cohort.stay_id,cohort.intime, chemistry.charttime, 'creatinine' as event, chemistry.creatinine as event_value
    from cohort
    inner join mimic_derived.chemistry chemistry
    on chemistry.hadm_id=cohort.hadm_id
    and cohort.intime<=chemistry.charttime
    where chemistry.creatinine is not null
union all
--SOFA
    select cohort.stay_id, cohort.intime, sofa_.endtime, 'sofa' as event, sofa_.sofa_24hours as event_value
    from cohort
    inner join mimic_derived.sofa sofa_
    on sofa_.stay_id=cohort.stay_id
    and cohort.intime<=sofa_.endtime
    where sofa_.sofa_24hours is not null
--enzyme
union all
    select cohort.stay_id, cohort.intime, enzyme.charttime, 'alt' as event, enzyme.alt as event_value
    from cohort
    inner join mimic_derived.enzyme enzyme
    on enzyme.hadm_id=cohort.hadm_id
    and cohort.intime<=enzyme.charttime
    where enzyme.alt is not null
union all
    select cohort.stay_id, cohort.intime, enzyme.charttime, 'ast' as event, enzyme.ast as event_value
    from cohort
    inner join mimic_derived.enzyme enzyme
    on enzyme.hadm_id=cohort.hadm_id
    and cohort.intime<=enzyme.charttime
    where enzyme.ast is not null
) AS events
order by stay_id, intime, charttime
'''

In [ ]:
df = pd.read_sql(query, con=engine)